# Enable Virtual Environment For This Notebook.

### Activate Conda Environment

<b>`$ conda activate`</b>

### Install Or Upgrade necessary software for virtual environment.

<b>`$ sudo apt-get install --upgrade python3-pip`</b>

<b>`$ sudo pip3 install --upgrade virtualenv`</b>

<b>`$ sudo pip3 install --upgrade setuptools`</b>

Now we will go to the location of the directory, where we will create our virtual environment.

<b>`$ cd /media/mujahid7292/Data/GoogleDriveSandCorp2014/ML_With_TensorFlow_On_GCP/06. End_To_End_ML_With_TensorFlow_On_GCP/Week_3/Lab_4_Preprocessing_with_Cloud_Dataflow/Practice`</b>

### Deactivate conda environment

<b>`$ conda deactivate`</b>

### Create Virtual Environment

<b>`$ virtualenv Venv`</b>

### Activate newly created virtual environment

<b>`$ source Venv/bin/activate`</b>

<b>`$ (Venv) which python`</b>

<b>`$ (Venv) pip list`</b>

<b>`$ (Venv) pip3 install jupyter`</b>

## <p style='color:red'>After installing jupyter notebook in the virtual environment, open this notebook from virtual environment.</p>

In [1]:
%%writefile requirements.txt
numpy
pandas
absl-py==0.7
pyarrow==0.15.1
absl-py==0.7
apache-beam[gcp]==2.20.0

Overwriting requirements.txt


In [2]:
%%bash
pip3 install -r requirements.txt

In [ ]:
%%bash
pip3 list

In [ ]:
%%bash
which python

In [ ]:
%%bash
python --version

# Notebook <a href="https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/06_structured/">Link</a>

# Necessary import of python package

In [3]:
import os
import tensorflow as tf
import tensorflow_transform as tft
import apache_beam as beam
print('Tensorflow: {}'.format(tf.__version__))
print('Tensorflow Transform: {}'.format(tft.__version__))
print('Apache beam: {}'.format(beam.__version__))

Tensorflow: 2.2.0
Tensorflow Transform: 0.22.0
Apache beam: 2.20.0


Those above version which ran successfully in this notebook.

* Tensorflow: 2.1.0
* Tensorflow Transform: 0.21.2
* Apache beam: 2.20.0

# Python Variable

In [4]:
# change these to try this notebook out
ACCOUNT = 'student-00-b4ae67740424@qwiklabs.net'
SAC = 'qwick-jupyter-notebook-sac-e'
SAC_KEY_DESTINATION = '/media/mujahid7292/Data/Gcloud_Tem_SAC'
BUCKET = 'qwiklabs-gcp-00-008dc3c1bc37'
PROJECT = 'qwiklabs-gcp-00-008dc3c1bc37'
REGION = 'us-central1'

# Bash Variable

In [5]:
os.environ['ACCOUNT'] = ACCOUNT
os.environ['SAC'] = SAC
os.environ['SAC_KEY_DESTINATION'] = SAC_KEY_DESTINATION
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# LogIn To Google Cloud

In [8]:
%%bash
gcloud auth login $ACCOUNT --force

Opening in existing browser session.


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=AdmmEQACviKMuKsVNUi5efvDwj55-ZZ8ncLA5Mzv0Ak&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


[0601/181040.922576:ERROR:nacl_helper_linux.cc(308)] NaCl helper process running without a sandbox!
Most likely you need to configure your SUID sandbox correctly

You are now logged in as [student-00-b4ae67740424@qwiklabs.net].
Your current project is [qwiklabs-gcp-04-7e2ebaba2884].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


# Set Current Project

In [9]:
%%bash
gcloud config set project $PROJECT

Updated property [core/project].


# Create Servic Account For Jupyter Notebook

In [10]:
%%bash
gcloud iam service-accounts create $SAC \
    --description "This service account will help us to connect local jupyter notebook with server" \
    --display-name "jupyter-sac"

Created service account [qwick-jupyter-notebook-sac-e].


# See the list of service account

In [11]:
%%bash
gcloud iam service-accounts list

NAME                                    EMAIL                                                                              DISABLED
jupyter-sac                             qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com  False
Compute Engine default service account  438107741394-compute@developer.gserviceaccount.com                                 False
App Engine default service account      qwiklabs-gcp-00-008dc3c1bc37@appspot.gserviceaccount.com                           False
Qwiklabs User Service Account           qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com  False


# Enabling The Service Account

In [12]:
%%bash
gcloud iam service-accounts enable $SAC@$PROJECT.iam.gserviceaccount.com --project $PROJECT

Enabled service account [qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com].


# Create New Key For Above Service Account

In [13]:
%%bash
gcloud iam service-accounts keys create $SAC_KEY_DESTINATION/$SAC.json \
  --iam-account $SAC@$PROJECT.iam.gserviceaccount.com \
  --project $PROJECT

created key [4d50de1e1514aa65265b61e5da0451fb5ccc1652] of type [json] as [/media/mujahid7292/Data/Gcloud_Tem_SAC/qwick-jupyter-notebook-sac-e.json] for [qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com]


It may take up to 60 seconds before a newly created key can be used for authentication. If you experience authentication failures immediately after creating a new key, ensure that 60 seconds have elapsed before trying again.

# Activate the service account with above key

In [14]:
%%bash
gcloud auth activate-service-account \
--key-file=${SAC_KEY_DESTINATION}/${SAC}.json

Activated service account credentials for: [qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com]


# Set Google Application Credentials

In [15]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='{}/{}.json'.format(SAC_KEY_DESTINATION,SAC)

Check Whether Google Application Credential Was Set Successfully Outside Virtual Environment

In [16]:
%%bash
set | grep GOOGLE_APPLICATION_CREDENTIALS 

GOOGLE_APPLICATION_CREDENTIALS=/media/mujahid7292/Data/Gcloud_Tem_SAC/qwick-jupyter-notebook-sac-e.json


# Set Default Project And Region

In [17]:
%%bash
gcloud config set account $ACCOUNT
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/account].
Updated property [core/project].
Updated property [compute/region].


# Give BigQuery Access Permision To This Service Account

In [20]:
%%bash
gcloud projects add-iam-policy-binding $PROJECT \
    --member serviceAccount:$SAC@$PROJECT.iam.gserviceaccount.com \
    --role roles/bigquery.admin

bindings:
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/appengine.appAdmin
- members:
  - serviceAccount:service-438107741394@gae-api-prod.google.com.iam.gserviceaccount.com
  role: roles/appengineflex.serviceAgent
- members:
  - serviceAccount:qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.admin
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataEditor
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquer

Updated IAM policy for project [qwiklabs-gcp-00-008dc3c1bc37].


# Give GCS Access Permision To This Service Account

In [18]:
%%bash
gcloud projects add-iam-policy-binding $PROJECT \
    --member serviceAccount:$SAC@$PROJECT.iam.gserviceaccount.com \
    --role roles/storage.admin

bindings:
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/appengine.appAdmin
- members:
  - serviceAccount:service-438107741394@gae-api-prod.google.com.iam.gserviceaccount.com
  role: roles/appengineflex.serviceAgent
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.admin
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataEditor
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataOwner
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.i

Updated IAM policy for project [qwiklabs-gcp-00-008dc3c1bc37].


# Give AI Platform Access Permision To This Service Account

In [19]:
%%bash
gcloud projects add-iam-policy-binding $PROJECT \
    --member serviceAccount:$SAC@$PROJECT.iam.gserviceaccount.com \
    --role roles/ml.admin

bindings:
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/appengine.appAdmin
- members:
  - serviceAccount:service-438107741394@gae-api-prod.google.com.iam.gserviceaccount.com
  role: roles/appengineflex.serviceAgent
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.admin
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataEditor
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataOwner
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.i

Updated IAM policy for project [qwiklabs-gcp-00-008dc3c1bc37].


## Give Dataflow Access Permission To This Service Account

In [21]:
%%bash
gcloud projects add-iam-policy-binding $PROJECT \
    --member serviceAccount:$SAC@$PROJECT.iam.gserviceaccount.com \
    --role roles/dataflow.admin

bindings:
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/appengine.appAdmin
- members:
  - serviceAccount:service-438107741394@gae-api-prod.google.com.iam.gserviceaccount.com
  role: roles/appengineflex.serviceAgent
- members:
  - serviceAccount:qwick-jupyter-notebook-sac-e@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.admin
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquery.dataEditor
- members:
  - serviceAccount:qwiklabs-gcp-00-008dc3c1bc37@qwiklabs-gcp-00-008dc3c1bc37.iam.gserviceaccount.com
  - user:student-00-b4ae67740424@qwiklabs.net
  role: roles/bigquer

Updated IAM policy for project [qwiklabs-gcp-00-008dc3c1bc37].


# Create GCS Bucket 

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} -p ${PROJECT} gs://${BUCKET}
  #gsutil mb -l us-east1 -p qwiklabs-gcp-02-3fc87b630759 gs://bucket-qwiklabs-gcp-02-3fc87b630759
fi